<b> Date written: 11/22/2021</b>

<b> Last modified: 11/24/2021</b>

The model contained in this notebook is a 1D river profile evolution model across multiple lithologies. Without being specific to actual rock type, the lithologies represented in this model will simply be referred to as "hard" and "soft" (though it should be noted that some values used in this model are experimental data from granites and sandstones, respectively). This model is applicable anywhere that rivers traverse multiple bedrock types, but was developed for the specific case of erosion in a sedimentary basin adjacent to a crystalline mountain range. 

Here we represent only two lithologies, but the code to set up the bedrock divisions could easily be altered to include more units. However, doing so should require careful thought about how each unit contributes (or not) to coarse bedload material in transport.

The basic functions of this model are:

- Erode bedrock via both plucking and abrasion
- Generate coarse sediment through plucking of resistant lithology
- Allow sediment fining downstream via grain attrition
- Modulate bedrock erosion rates based on amount of aslluvial bed cover present

This resource will be updated with more figures, examples, animations, annotations, and other code improvements in the near future. This is just a first contribution ahead of AGU 2021.

Planned updates:
- Add plotting and animnation to fuction (<i>Purpose: automatically generate animation everytime function is run, and easily capture outputs from different timesteps without having to re-run function</i>)
- Write `q` as an integral-style sum (<i> Purpose: honor that upstream increases in runoff also affect downstream discharge, even if downstream runoff does not increase </i>)
- Add more annotations within function (<i>Purpose: additional explanation for new user</i>)
- Try to write loop part of function <i>outside</i> of function first, then call it within the function (<i>Purpose: streamline code, make it easier to read with less scrolling necessary to see full function</i>

See below table for key to variable names.

| variable name | meaning | units | data type |
| --- | --- | --- | --- |
| dx | grid spacing | m | int or float | 
| x | grid length | m | array |
| r | runoff rate | m/yr | int or float |
| Hstar | characteristic sediment thickness (think of as scour depth) | m | int or float |
| H | sediment thickness | m | array of length `x` |
| etab | bedrock elevation | m | array of length `x` |
| eta | total topographic elevation | m | array of length `x` |
| k_small | erodibility of resistant lithology | yr$^{-1}$ | int |
| k_large | erodibility of soft lithology | yr$^{-1}$ | int |
| k_hard | bedrock erodibility along profile (hard rocks) | yr$^{-1}$ | array of length `x`, filled with `k_small` over areas of resistant rocks and filled with zeros over areas of soft rocks |
| k_soft | bedrock erodibility along profile (soft rocks) | yr$^{-1}$ | array of length `x`, filled with `k_large` over areas of soft rocks and filled with zeros over areas of hard rocks |
| beta_small | abrasion coefficient for resistant lithology (average of granite abrasion values from Attal and Lave 2006, converted to % mass loss per m) | m$^{-1}$ | int |
| beta_large | abrasion coeffcient for soft lithology (average of sandstone abrasion values from Attal and Lave 2006, converted to % mass loss per m) | m$^{-1}$ | int |
| beta_hard | abrasion coefficient array for hard rocks | m$^{-1}$ | array of length `x`, filled with `beta_small` over areas of resistant rocks and filled with zeros over areas of soft rocks |
| beta_soft | abrasion coefficient array for soft rocks | m$^{-1}$ | array of length `x`, filled with `beta_large` over areas of resistant rocks and filled with zeros over areas of soft rocks |
|atr_factor | abrasion coeffcient relevant for grain attrition. Since we assume all bedload is derived from "hard rock," this value should be the same as the abrasion coefficient `beta_small` | m$^{-1}$ | int |


In [1]:
# START BY IMPORTING LIBRARIES
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# SET UP GRID
dx = 1000
x = np.arange(0, 100000, dx)

# SET UP RUNOFF RATE
r = np.zeros(len(x))
r[:] = 1

# SET UP REFERENCES TO TOPOGRAPHY
Hstar = 1
H = Hstar + np.zeros(len(x))
etab = -H
eta = etab + H

# SET BEDROCK ERODIBILITY VALUES ACROSS GRID
k_small = 0.0001
k_large = 0.001

k_hard = np.zeros(len(x))
k_hard[:25] = k_small

k_soft = np.zeros(len(x))
k_soft[25:] = k_large

# SET ABRASION COEFFICIENTS
beta_small = 0.000004
beta_large = 0.000064

beta_hard = np.zeros(len(x))
beta_hard[:25] = beta_small

beta_soft = np.zeros(len(x))
beta_soft[25:] = beta_large

atr_factor = beta_small

In [ ]:
# write a function that has two bedrock lithologies in the domain, but only one contributing sediment to bedload
# give this function whatever name you like

def two_lith_one_sed(x,
                     dx,
                     Hstar,
                     H,
                     etab,
                     eta,
                     k_hard,
                     k_soft,
                     beta_hard,
                     beta_soft,
                     atr_factor,
                     r,
                     c = 1,
                     baselevel_rate = 0.001,
                     num_steps = 1000000, 
                     porosity = 0.55):
    
    H[-1] = 0.0
    bedrock_ero = np.zeros(len(x))  # bedrock erosion rate
    sedimentation_rate = np.zeros(len(x))
    total_ero = np.zeros(len(x))
    total_ero[-1] = baselevel_rate
    q = r * x  # discharge = distance downstream (first node is left edge of 0th cell)
    qs = np.zeros(len(x))  # first node is left edge of 0th cell
    dt_global = 0.2 * dx * dx / (c*q[-1])  # "global" time-step size
    run_duration = dt_global * num_steps  # <== here's how long we want to run
    cum_time = 0.0  # <== keep track of elapsed time

    #for i in range(num_steps):
    while cum_time < run_duration:  # <== use a while loop because dt varies by iteration
        

        # first calculate rates
        
        #  calc slope
        S = -np.diff(eta)/dx
        
        #  calculate e factor
        efac = np.exp(- H / Hstar)
        
        #  calculate total bedload sed flux and set boundary condition
        qs[1:] = c * q[1:] * S * (1.0 - efac[:-1])
        qs[0] = 0
        
        #  calc bedrock erosion from stream power (plucking)
        ero_plucking_ig = efac[:-1] * (k_ig[1:] * q[1:] * S)
        ero_plucking_sed = efac[:-1] * (k_sed[1:] * q[1:] * S)
        
        #  calc bedrock erosion from abrasion
        ero_ab_ig = efac[:-1] * (beta_ig[:-1] * qs[1:])   # <== change indexing: qs[1] represents node 0
        ero_ab_sed = efac[:-1] * (beta_sed[:-1] * qs[1:])
        
        #  calc bedrock erosion rate from stream power and abrasion
        bedrock_ero[:-1] = ero_plucking_ig + ero_plucking_sed + ero_ab_ig + ero_ab_sed
        
        #  calc grain attrition rate
        atr = atr_factor * qs[1:]
        
        #  calc rate of change in alluvial thickness
        sedimentation_rate[:-1] = -((1 / porosity) * ((np.diff(qs)/dx) + atr - ero_plucking_ig))
        
        
        # Calculate maximum allowable time-step size
        
        #  track total erosion rate
        total_ero[:-1] = bedrock_ero[:-1] - sedimentation_rate[:-1]  # <== erosion is MINUS sed rate
        
        #  set adaptive timestep
        #  first check time to flat surface
        elev_diff = np.diff(eta)/dx
        ero_diff = np.diff(total_ero)/dx
        #valid_places = np.where(ero_diff < 0)
        valid_places = np.where(ero_diff < 0)[0]  # <== we just want the array, not the full tuple from where()
        if len(valid_places) > 0:  # <== in case there ARE no locations...
            times_to_flat = np.abs(elev_diff[valid_places]/ero_diff[valid_places])
        else:
            times_to_flat = np.array([dt_global])  # <== ...we just revert to the global dt
        min_time_to_flat = np.amin(times_to_flat)

        #  then check time to deplete all sediment
        #sed_depletion_locations = np.where(sedimentation_rate < 0)
        sed_depletion_locations = np.where(sedimentation_rate < 0)[0]  # <== we just want the array, not the full tuple from where()
        if len(sed_depletion_locations) > 0:  # <== in case there ARE no locations...
            times_to_no_sed = np.abs(H[sed_depletion_locations]/sedimentation_rate[sed_depletion_locations])
        else:
            times_to_no_sed = np.array([dt_global])  # <== ...we just revert to the global dt
        min_time_to_no_sed = np.amin(times_to_no_sed)

        #  check for smaller condition
        dt = min(min_time_to_flat, min_time_to_no_sed)

        #  if larger than global step size, limit to global
        dt = min(dt, dt_global)
        
        
        # Update quantities
        
        #  lower baselevel
        eta[-1] -= baselevel_rate * dt 
        
        #  set boundary conditions
        etab[-1] = eta[-1]
        
        #  calc change in bedrock elev
        etab[:-1] -= bedrock_ero[:-1] * dt
        
        #  update sediment thickness
        H[:-1] += sedimentation_rate[:-1] * dt
        H[H < 0] = 0

        #  update elev
        eta[:-1] = etab[:-1] + H[:-1]
        
        # Advance time
        cum_time += dt
        
        if any(total_ero[:] != baselevel_rate):
            continue
        else:
            break
            
    print(cum_time)
        
    return (S, qs, efac, eta, etab, ero_plucking_ig, ero_plucking_sed, ero_ab_ig, ero_ab_sed, bedrock_ero, atr, 
            sedimentation_rate, H, total_ero, cum_time)